# Image Effects

My attempt at recreating trivial photoshop effects + some custom visualizations I find cool.

## Imports

Let's get these out of the way.

In [ ]:
%load_ext autoreload
%autoreload 2

import textwrap
import math
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
import os
from scipy.ndimage import gaussian_filter
from src.utils import load_rgb, save_gray, adjust_exponential, plot
import cv2
from sklearn.cluster import KMeans

## Grayscale

One might think grayscale is as easy as averaging the rgb channels but our eyes
perceive things differently. Thus a better way to grayscale is to use non-linear
weights when combining the rgb channels.

More here: https://en.wikipedia.org/wiki/Grayscale

In [ ]:
rgb = load_rgb("img/lenna.png")
gray = np.dot(rgb[...,:3], [0.299, 0.587, 0.114])
plot(1, 2, ("Original", rgb), ("Gray", gray))

## Hue shifting

Load image as hsv then shift all colors using a simple offset

In [ ]:
# Load colorful image.
rgb = load_rgb('img/food.jpg')
hsv = cv2.cvtColor(rgb, cv2.COLOR_RGB2HSV)

# Created shifted image.
hue = hsv[..., 0]
shift = 90
shifted_hue = (hue + shift) % 180
hsv[:, :, 0] = shifted_hue
shifted_rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)

# Plot comparison.
plot(1, 2,
    ("Original", rgb),
    ("Shifted Hue", shifted_rgb),
    figsize=(5, 5)
)

## Modeling macos dynamic backgrounds

Use gamma correction to emulate macos dynamic backgrounds.

In [ ]:
rgb = load_rgb("img/beach.jpg")
gammas = [1.3, 1, 0.7, 0.4]

divides = np.linspace(0, rgb.shape[1], 5).astype(int)
for start, stop, gamma in zip(divides, divides[1:], gammas):
    rgb[:,start:stop, :] = adjust_exponential(rgb[:,start:stop, :], gamma)

plot(1, 1, ("Modeling macos dynamic backgrounds", rgb))

## Shifting saturation

Description: Same concept as previous demo but messing with saturation instead of gamma

In [ ]:
rgb = load_rgb("img/beach.jpg")
hsv_exp = cv2.cvtColor(rgb, cv2.COLOR_RGB2HSV)
hsv_lin = cv2.cvtColor(rgb, cv2.COLOR_RGB2HSV)

saturations = [0.4, 0.7, 1, 1.3]
divides = np.linspace(0, hsv_exp.shape[1], len(saturations) + 1).astype(int)

for start, stop, saturation in zip(divides, divides[1:], saturations):
    hsv_lin[:,start:stop, 1] = hsv_lin[:,start:stop, 1] * saturation
    hsv_exp[:,start:stop, 1] = adjust_exponential(hsv_exp[:,start:stop, 1], saturation)

# Display.
plot(1, 3,
    ("rgb", rgb),
    ("Linear", cv2.cvtColor(hsv_lin, cv2.COLOR_HSV2RGB)),
    ("Exponential", cv2.cvtColor(hsv_exp, cv2.COLOR_HSV2RGB)),
    figsize=(10, 10)
)

## Color quantization + pixelation

Use color quantization + pixelation to convert new mario photo into old 1980s version

In [ ]:
# Load image.
rgb = load_rgb('img/mario.png')

# Create pixelated image.
orig_h, orig_w = rgb.shape[:2]
pixel_w, pixel_h = (25, 38)
down_sampled = cv2.resize(rgb, (pixel_w, pixel_h), interpolation=cv2.INTER_LINEAR)
pixelated = cv2.resize(down_sampled, (orig_w, orig_h), interpolation=cv2.INTER_NEAREST)

# Quantize the pixelated image.
n_colors = 7
arr = pixelated.reshape((-1, 3))
kmeans = KMeans(n_clusters=n_colors, n_init="auto").fit(arr)
labels = kmeans.labels_
centers = kmeans.cluster_centers_
quantized_pixelated = centers[labels].reshape(pixelated.shape).astype('uint8')

# Show original, pixelated, quantized_pixelated.
plot(1, 3,
    ("Original", rgb),
    ("Pixelated", pixelated),
    ("Quantized+Pixelation", quantized_pixelated),
)

## Reflection

In [ ]:
rgb = load_rgb("img/high-five.png")
reflected = cv2.flip(rgb, 1)
joined = np.concatenate((rgb, reflected), axis=1)

plot(1,1,
    ("Original + Reflected", joined),
    figsize=(3, 3)
)

## Median vs Gaussian blurs on noise reduction

Both median and gaussian blur operations smooth out images but have different
effects. Median blurring works by looking at every cell and taking the median of
it and all neighboring cells. Gaussian blurring works by looking at every cell and
averaging it with its neighboring cells.

One benefit of median blurring is its good at getting rid of noise instead of
averaging it in.

In [ ]:
rgb = load_rgb("img/wink.png")
noise = np.random.choice([0, 255], size=rgb.shape, p=[0.9, 0.1])
noisy_rgb = (noise + rgb).astype('uint8')

plot(2,2,
    ("Original", rgb),
    ("With noise", noisy_rgb),
    ("Median", cv2.medianBlur(noisy_rgb, 5)),
    ("Gaussian", cv2.GaussianBlur(noisy_rgb, (5,5), 0))
)

## Erode and Dilate to fill in holes

This demo will load an image of a dice, find the dice, then fill in the holes
by enlarging and shrinking the mask of the dice.

In [ ]:
# Load dice and convert to hsv.
rgb = load_rgb("img/dice.png")
hsv = cv2.cvtColor(rgb, cv2.COLOR_RGB2HSV)

# Select all the red to find the dice.
lower = np.array([160, 0, 0], dtype="uint8")
upper = np.array([185, 255, 255], dtype="uint8")
mask = cv2.inRange(hsv, lower, upper)

# Fill in the holes by enlarging then shrinking the mask.
kernel = np.ones((20, 20), np.uint8)
dilated = cv2.dilate(mask, kernel, iterations=1)
eroded = cv2.erode(dilated, kernel, iterations=1)

# Display.
plot(2, 2,
    ("Original", rgb),
    ("Mask", mask),
    ("Dilated", dilated),
    ("Eroded", eroded),
    figsize=(3,3)
)

## Rotate a caret to create star

In [ ]:
bgra = cv2.imread("img/star_part.png", flags=cv2.IMREAD_UNCHANGED)
rgba = cv2.cvtColor(bgra, cv2.COLOR_BGR2RGBA)

image_center = tuple(np.array(rgba.shape[1::-1]) / 2)

star = rgba.copy()
for i in range(1, 5):
    rot_mat = cv2.getRotationMatrix2D(image_center, 72*i, 1.0)
    rotated = cv2.warpAffine(rgba, rot_mat, rgba.shape[1::-1], flags=cv2.INTER_LINEAR)
    star += rotated

plot(1, 2,
    ("Original", rgba),
    ("5 rotations merged", star),
    figsize=(5,5)
)

## Shifting portions of an image around

In [ ]:
rgb = load_rgb("img/pineapple.jpg")
shifted = np.zeros(rgb.shape, dtype=np.uint8)

shift_scalar = 40
height, width = rgb.shape[:-1]
for i in range(height):
    delta_x = int(shift_scalar * math.sin(math.radians(2*i)))
    insert_x = max(0, delta_x)
    insert_x2 = min(width, width + delta_x)
    extract_x1 = max(0, -delta_x)
    extract_x2 = min(width, width - delta_x)
    shifted[i, insert_x:insert_x2] = rgb[i, extract_x1:extract_x2]

plot(1, 2,
    ("Original", rgb),
    ("Shifted", shifted),
    figsize=(4,4)
)

## Noise gradient


In [ ]:
rgb = load_rgb("img/pineapple.jpg")
noise_gradient = np.zeros(rgb.shape, dtype=np.uint8)

noise_scalar = 0.5
height, width = rgb.shape[:-1]
for i in range(height):
    noise = np.random.normal(0, i * noise_scalar, (width, 3))
    noise_gradient[i] = (rgb[i] + noise).clip(0, 255).astype(np.uint8)

plot(1, 2,
    ("Original", rgb),
    ("Noise Gradient", noise_gradient),
    figsize=(4,4)
)

## Bloom effect

Add a halo in an image and make it glow

- Add halo in image
- Blur mask of halo
- Blend halo back in

In [ ]:
rgb = load_rgb("img/person.png")

# Create halo.
halo = np.zeros(shape=rgb.shape, dtype=np.uint8)
center = (halo.shape[0] // 2, 9)
halo_size = (40, 7)
angle = 0
start_angle = 0
end_angle = 360
thickness = 2
color = (255, 223, 50)
halo = cv2.ellipse(halo, center, halo_size, angle, start_angle, end_angle, color, thickness)
halo = cv2.GaussianBlur(halo, (5, 5), sigmaX=3)

# Blend the layers together.
gain = 2
overlaid_halo = cv2.addWeighted(rgb, 1, halo, gain, 0)

# Display.
plot(1, 3,
    ("Original", rgb),
    ("Halo", halo),
    ("Overlaid", overlaid_halo),
    figsize=(4,4)
)

## Additive color mixing

Tvs on a per pixel basis are actually made of three separate sub-pixels.
The sub-pixels comprising of red, green, blue.
I'm curious what recreating this effect looks like if I make an image pixel
the subpixel and consider every three rows a pixel.

I explore 2 different methods here:

1. Make every row either red, green or blue depending on row mod 3x
2. Resize by a factor of 3 then merge

The result is lame lol. I'm essentially just throwing away 2/3 of the data but
it was fun to explore.

In [ ]:
rgb = load_rgb("img/parrot.jpg")
r, g, b = cv2.split(rgb)

# Method 1: zeroing out channels.
r[0::3] = 0
r[1::3] = 0
g[1::3] = 0
g[2::3] = 0
b[2::3] = 0
b[0::3] = 0
method_1 = cv2.merge((r, g, b))

# Method 2: downsample then upsample selectively.
h, w = rgb.shape[:-1]
down_sampled_rgb = cv2.resize(rgb, (w, h//3))
down_sampled_r, down_sampled_g, down_sampled_b = cv2.split(down_sampled_rgb)
method_2 = np.zeros(rgb.shape, dtype=rgb.dtype)
method_2[0::3, :, 0] = down_sampled_r
method_2[1::3, :, 1] = down_sampled_g
method_2[2::3, :, 2] = down_sampled_b

# Display.
plot(1, 3,
    ("Original", rgb),
    ("Method 1", method_1),
    ("Method 2", method_2),
    figsize=(5, 5)
)

## Cut out text overlay effect

In [ ]:

rgb = load_rgb("img/fashion_portrait.jpg")
height, width = rgb.shape[:-1]

def create_text_mask(fontFace, fontScale):
    lorem_ipsum = ("Lorem ipsum dolor sit amet, consectetur adipiscing elit, "
        "sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. "
        "Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris "
        "nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in "
        "reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla "
        "pariatur. Excepteur sint occaecat cupidatat non proident, sunt in "
        "culpa qui officia deserunt mollit anim id est laborum."
    )
    wrapped_text = textwrap.wrap(lorem_ipsum, width=35)
    text_mask = np.zeros(rgb.shape, np.uint8)
    for i, line in enumerate(wrapped_text, start=1):
        cv2.putText(
            text_mask,
            text=line,
            org=(0, i*40),
            fontFace=fontFace,
            fontScale=fontScale,
            color=(255,255,255),
            thickness=3
        )
    return text_mask

# Create text masks.
text_mask_1 = create_text_mask(7, 3)
text_mask_2 = create_text_mask(0, 1)

# Rotate second text mask.
image_center = tuple(np.array((height, width)) / 2)
rot_mat = cv2.getRotationMatrix2D(image_center, 90, 1.0)
text_mask_2 = cv2.warpAffine(text_mask_2, rot_mat, (height, width), flags=cv2.INTER_LINEAR)

# Combine text masks then use that as a mask for the rgb image.
text_mask = cv2.bitwise_or(text_mask_1, text_mask_2)
combined = cv2.bitwise_and(rgb, text_mask)

# Display.
plot(1, 3,
    ("Original", rgb),
    ("Text", text_mask),
    ("Combined", combined),
    figsize=(7, 7)
)

## Find tennis ball


To find the tennis ball:

- Look for it's approximate colors
- Try to filter out unwanted colors
- Find the contour around the blobs found in the mask
- Find the minimum circle around the biggest blob and consider it the ball

Note: the values for the threshold we're very precisely tuned


In [ ]:
rgb = load_rgb("img/tennis_ball.jpg")
smoothed_rgb = cv2.GaussianBlur(rgb, (11, 11), 0)
hsv = cv2.cvtColor(smoothed_rgb, cv2.COLOR_RGB2HSV)

# Find pixels likely belonging to tennis ball.
lower = np.array((34, 152, 132), dtype="uint8")
upper = np.array((44, 255, 255), dtype="uint8")
threshold = cv2.inRange(hsv, lower, upper)
eroded = cv2.erode(threshold, np.ones((5, 5), np.uint8), iterations=3)
dilated = cv2.dilate(eroded, np.ones((5, 5), np.uint8), iterations=4)

# Find tennis ball from probable tennis ball pixels.
contours, _ = cv2.findContours(dilated.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
rgb_copy = rgb.copy()
cv2.drawContours(rgb_copy, contours, -1, 255, 3)
(x,y),radius = cv2.minEnclosingCircle(contours[0])
center = (int(x),int(y))
radius = int(radius)
cv2.circle(rgb_copy, center,radius,(0,255,0),2)

# Display.
plot(1,6,
    ("Original", rgb),
    ("Smoothed", smoothed_rgb),
    ("Threshold", threshold),
    ("Eroded", eroded),
    ("Dilated", dilated),
    ("Found", rgb_copy),
    figsize=(7,7)
)

## Finding the horizon using a canny edge detector

For this demo, I wanted to explore using a canny edge detector to separate the
ground from the sky. Took a bit of work to find the right settings for the gaussian
filter and canny hysteresis threshold.

The gaussian filter is used to weed out weak edges in order to isolate the horizon.

The [hysteresis threshold](https://theailearner.com/tag/hysteresis-thresholding/)
controls how the edge detector connects weak edges to strong edges.

In [ ]:
rgb = load_rgb("img/sunset.jpg")
gaussian = cv2.GaussianBlur(rgb, (21, 21), 0)

# Explore canny edge detectors w/ and w/o smoothing.
t1 = 100
t2 = 240
canny_rgb = cv2.Canny(image=rgb, threshold1=t1, threshold2=t2)
canny_gaussian = cv2.Canny(image=gaussian, threshold1=t1, threshold2=t2)

# Create new image.
separated = np.zeros(rgb.shape, dtype=np.uint8)
top_color = (255,0,0)
middle_color = (255,255,255)
bottom_color = (0,0,255)
thickness = 30
for col_index, col in enumerate(canny_gaussian.T):
    sep = int(np.where(col == 255)[0].mean())
    separated[:sep-thickness,col_index,:] = top_color
    separated[sep-thickness:sep+thickness,col_index,:] = middle_color
    separated[sep+thickness + 1:,col_index,:] = bottom_color

# Display.
plot(1,5,
    ("RGB", rgb),
    ("Gaussian", gaussian),
    ("Canny RGB", canny_rgb),
    ("Canny Gaussian", canny_gaussian),
    ("Horizon", separated),
    figsize=(7,7)
)